# Lesson 13.1 深度学习建模目标与性能评估理论

&emsp;&emsp;从Lesson 13起，我们讲开始系统介绍深度学习建模理论，以及建模过程中的优化方法。

## 一、在Jupyter初始化过程中自动加载常用包的设置方法

&emsp;&emsp;在每一节课程的开头，我们都要导入常用包，由于这项工作重复而固定，因此我们也可以通过配置jupyter（准确来说应该是ipython）的startup文件，来使得每次新创建一个ipy文件时，都能够自动加载配置好的包，从而就能免去每节开头导入包的相关操作。（所以说，懒惰是推动技术进步的不竭动力。）相关方法如下：

- 找到startup文件夹

&emsp;&emsp;在当前用户主目录下，找到`.ipython`文件夹，然后进入到`profile_default`文件夹内，并找到`startup`文件夹。此处如果profile_default内没有startup文件夹，自己新建一个即可。

<img src="https://i.loli.net/2021/02/21/8ZXCNOWezrHoaEy.jpg" alt="55" style="zoom:30%;" />

- 创建start.py文件

&emsp;&emsp;接下来，在startup文件夹内，创建一个start.py文件。关于创建py文件的方法此前介绍过，此处我们只需先创建一个txt文件，然后将其名称和后缀改为start.py即可。

<img src="https://i.loli.net/2021/02/21/pS81Ht6vDUfd7L4.jpg" alt="56" style="zoom:30%;" />

<img src="https://i.loli.net/2021/02/21/aRSxEuTCOlo9UHV.jpg" alt="57" style="zoom:50%;" />

- 输入每次初始化时需要执行的代码

&emsp;&emsp;在start.py中输入每次初始化时导入包的代码，相关代码如下：

> 注意，上述cell是raw模式，不是代码模式。

我们可以使用记事本打开.py文件，然后复制上述内容然后保存。

<img src="https://i.loli.net/2021/02/21/bXWaUhZTJMG7PQz.jpg" alt="58" style="zoom:30%;" />

- 重启ipy，检测是否生效

然后需要重启ipy kernel，此处重启的方法也非常简单，只需在jupyter中先将当前kernel关闭

<img src="https://i.loli.net/2021/02/21/ORbHfrPWxASEMvZ.jpg" alt="59" style="zoom:33%;" />

在最左侧栏中选择文件目录的下一个按钮，查看当前运行的kernel和terminal，找到Lesson 13.1（当前打开的ipy文件），然后点击SHUT DOWN。此时当前页面右上角就会显示为No Kernel，点击选择Python 3，就相当于重启了当前jupyter中的Python kernel。

<img src="https://i.loli.net/2021/02/21/VdMP52LoZIjEAD4.jpg" alt="60" style="zoom:33%;" />

<img src="https://i.loli.net/2021/02/21/aNnWuFBRtKeZA6y.jpg" alt="61" style="zoom:33%;" />

当然，也可以直接点击重启按钮，重启当前jupyter kernel。

<img src="https://i.loli.net/2021/02/26/AqXGEdvliUw154t.jpg" alt="65" style="zoom:50%;" />

- 测试初始化配置是否生效

In [1]:
torch.tensor(1)

tensor(1)

至此也验证了初始化设置成功，每次创建jupyter文件时都将自动导入我们设置好的第三方库，即可免去每节开始的导包代码。

## 二、机器学习目标与模型评估方法

&emsp;&emsp;在了解深度学习基本模型的概念与实现方法后，接下来，我们将详细探讨深度学习模型优化的常用方法。从上一课的实验中不难发现，要把一个模型“建好”已是不容易，而要想办法把模型的效果进行提升，如果没有基础理论支持和方法论工具，优化过程无异于盲人摸象。因此，本节课将从建模的根本目标出发，围绕模型优化的基本概念和核心理论进行全面梳理，并在此基础之上介绍相关实践方法，逐渐拨开模型优化面前的迷雾。

&emsp;&emsp;我们经常会对模型的好坏优劣进行评估，Lesson 12中我们也使用准确率、MSE等指标评估建模结果，看起来模型评估是围绕某项指标在进行评估，指标好模型就好，指标不好模型就不好，其实并不完全如此。要了解模型的性能其实并不简单，固然我们会使用某些指标去进行模型评估，但其实指标也只是我们了解模型性能的途径而不是模型性能本身。而要真实、深刻的评判模型性能，就必须首先了解机器学习的建模目标，并在此基础之上熟悉我们判断模型是否能够完成目标的一些方法，当然，只有真实了解的模型性能，我们才能进一步考虑如何提升模型性能。因此，在正式讲解模型优化方法之前，我们需要花些时间讨论机器学习算法的建模目标、机器学习算法为了能够达到目标的一般思路，以及评估模型性能的手段，也就是模型评估指标。

&emsp;&emsp;无论是机器学习还是传统的统计分析模型，核心使命就是探索数字规律，而有监督学习则是希望在探索数字规律的基础上进一步对未来进行预测，当然，在数字的世界，这个预测未来，也就是预测未来某项事件的某项数值指标，如某地区未来患病人次、具备某种数字特征的图片上的动物是哪一类，此处的未来也并非指绝对意义上的以后的时间，而是在模型训练阶段暂时未接触到的数据。正是因为模型有了在未知标签情况下进行预判的能力，有监督学习才有了存在的价值，但我们知道，基本上所有的模型，都只能从以往的历史经验当中进行学习，也就是在以往的、已经知道的数据集上进行训练（如上述利用已知数据集进行模型训练，如利用过往股票数据训练时间序列模型），这里的核心矛盾在于，在以往的数据中提取出来的经验（也就是模型），怎么证明能够在接下来的数据中也具备一定的预测能力呢？或者说，要怎么训练模型，才能让模型在未知的数据集上也拥有良好的表现呢？

&emsp;&emsp;目的相同，但在具体的实现方法上，传统的数理统计分析建模和机器学习采用了不同的解决方案。

&emsp;&emsp;首先，在统计分析领域，我们会假设现在的数据和未来的数据其实都属于某个存在但不可获得的总体，也就是说，现在和未来的数据都是从某个总体中抽样而来的，都是这个总体的样本。而正式因为这些数据属于同一个总体，因此具备某些相同的规律，而现在挖掘到的数据规律也就在某些程度上可以应用到未来的数据当中去，不过呢，不同抽样的样本之间也会有个体之间的区别，另外模型本身也无法完全捕获规律，而这些就是误差的来源。

&emsp;&emsp;虽然样本和总体的概念是统计学概念，但样本和总体的概念所假设的前后数据的“局部规律一致性”，却是所有机器学习建模的基础。试想一下，如果获取到的数据前后描绘的不是一件事情，那么模型训练也就毫无价值（比如拿着A股走势预测的时间序列预测某地区下个季度患病人次）。因此，无论是机器学习所强调的从业务角度出发，要确保前后数据描述的一致性，还是统计分析所强调的样本和总体的概念，都是建模的基础。

&emsp;&emsp;在有了假设基础之后，统计分析就会利用一系列的数学方法和数理统计工具去推导总体的基本规律，也就是变量的分布规律和一些统计量的取值，由于这个过程是通过已知的样本去推断未知的总体，因此会有大量的“估计”和“检验”，在确定了总体的基本分布规律之后，才能够进一步使用统计分析模型构建模型（这也就是为什么在数理统计分析领域，构建线性回归模型需要先进行一系列的检验和变换的原因），当然，这些模型都是在总体规律基础之上、根据样本具体的数值进行的建模，我们自然有理由相信这些模型对接下来仍然是从总体中抽样而来的样本还是会具备一定的预测能力，这也就是我们对统计分析模型“信心”的来源。简单来说，就是我们通过样本推断总体的规律，然后结合总体的规律和样本的数值构建模型，由于模型也描绘了总体规律，所以模型对接下来从总体当中抽样而来的数据也会有不错的预测效果，这个过程我们可以通过下图来进行表示。

<img src="https://i.loli.net/2021/02/05/KIpJCDyq9VQoliN.jpg" alt="37" style="zoom:40%;" />

&emsp;&emsp;而对于机器学习来说，并没有借助“样本-总体”的基本理论，而是简单的采用了一种后验的方法来判别模型有效性，前面说到，我们假设前后获取的数据拥有规律一致性，但数据彼此之间又略有不同，为了能够在捕捉规律的同时又能考虑到“略有不同”所带来的误差，机器学习会把当前能获取到的数据划分成训练集(trainSet)和测试集(testSet)，在训练集上构建模型，然后带入测试集的数据，观测在测试集上模型预测结果和真实结果之间的差异。这个过程其实就是在模拟获取到真实数据之后模型预测的情况，此前说到，模型能够在未知标签的数据集上进行预测，就是模型的核心价值，此时的测试集就是用于模拟未来的未知标签的数据集。如果模型能够在测试集上有不错的预测效果，我们就“简单粗暴”的认为模型可以在真实的未来获取的未知数据集上有不错的表现。其一般过程可以由下图表示。

<img src="https://i.loli.net/2021/02/05/fGzPxCQ1qoOZuFs.jpg" alt="38" style="zoom:40%;" />

&emsp;&emsp;虽然对比起数理统计分析，机器学习的证明模型有效性的过程更加“简单”，毕竟只要一次“模拟”成功，我们就认为模型对未来的数据也拥有判别效力，但这种“简单”的处理方式却非常实用，可以说，这是一种经过长期实践被证明的行之有效的方法。这也是为什么机器学习很多时候也被认为是实证类的方法，而在以后的学习中，我们也将了解到，机器学习有很多方法都是“经验总结的结果”。相比数理统计分析，确实没有“那么严谨”，但更易于理解的理论和更通用的方法，却使得机器学习可以在更为广泛的应用场景中发挥作用。（当然，负面影响却是，机器学习在曾经的很长一段时间内并不是主流的算法。）

&emsp;&emsp;据此，我们称模型在训练集上误差称为训练误差，在测试集上的误差称为泛化误差，不过毕竟在测试集上进行测试还只是模拟演习，我们采用模型的泛化能力来描述模型在未知数据上的判别能力，当然泛化能力无法准确衡量（未知的数据还未到来，到来的数据都变成了已知数据），我们只能通过模型在训练集和测试集上的表现，判别模型泛化能力，当然，就像此前说的一样，最基本的，我们会通过模型在测试集上的表现来判断模型的泛化能力。

## 三、手动实现训练集和测试集切分

&emsp;&emsp;接下来我们开始实践模型评估过程，首先是对训练集和测试集的划分，我们尝试创建一个切分训练集和测试集的函数。

In [4]:
def data_split(features, labels, rate=0.7):
    """
    训练集和测试集切分函数
    
    :param features: 输入的特征张量
    :param labels：输入的标签张量
    :param rate：训练集占所有数据的比例
    :return Xtrain, Xtest, ytrain, ytest：返回特征张量的训练集、测试集，以及标签张量的训练集、测试集 
    """
    num_examples = len(features)                              # 总数据量
    indices = list(range(num_examples))                       # 数据集行索引
    random.shuffle(indices)                                   # 乱序调整                     
    num_train = int(num_examples * rate)                      # 训练集数量 
    indices_train = torch.tensor(indices[: num_train])        # 在已经乱序的的indices中挑出前num_train数量的行索引值
    indices_test = torch.tensor(indices[num_train: ])         
    Xtrain = features[indices_train]                          # 训练集特征
    ytrain = labels[indices_train]                            # 训练集标签
    Xtest = features[indices_test]                            # 测试集特征
    ytest = labels[indices_test]                              # 测试集标签
    return Xtrain, Xtest, ytrain, ytest

> &emsp;&emsp;一般来说，训练集和测试集可以按照8：2或7：3比例进行划分。在进行数据划分的过程中，如果测试集划分数据过多，参与模型训练的数据就会相应减少，而训练数据不足则会导致模型无法正常训练、损失函数无法收敛、模型过拟合等问题，但如果反过来测试集划分数据过少，则无法代表一般数据情况测试模型是否对未知数据也有很好的预测作用。因此，根据经验，我们一般来说会按照8：2或7：3比例进行划分。      
&emsp;&emsp;看到这里，相信肯定有小伙伴觉得根据所谓的“经验”来定数据集划分比例不太严谨，有没有一种方法能够“精准”的确定什么划分比例最佳呢？例如通过类似最小二乘法或者梯度下降这类优化算法来计算划分比例？各位同学可以尝试着进行思考，并给出自己的答案。课程中将在下一节介绍参数和超参数时给出详细解答。         
&emsp;&emsp;值得一提的是，在机器学习领域，充斥着大量的“经验之谈”或者“约定俗成”的规则，一方面这些经验为建模提供了诸多便捷、也节省了很多算力，但另一方面，通过经验来决定影响模型效果的一些“超参数”取值的不严谨的做法，也被数理统计分析流派所诟病。

接下来，测试函数性能

In [5]:
f = torch.arange(10)                # 创建特征0-9
f

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:
l = torch.arange(1, 11)             # 创建标签1-10，保持和特征+1的关系
l

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [7]:
data_split(f, l)

(tensor([1, 6, 4, 5, 8, 3, 0]),
 tensor([9, 7, 2]),
 tensor([2, 7, 5, 6, 9, 4, 1]),
 tensor([10,  8,  3]))

接下来，还是在上一节课的内容上，尝试带入训练集进行建模，利用测试集评估模型建模效果

In [1]:
# 设置随机数种子
torch.manual_seed(420)   

# 生成回归类数据集
features, labels = tensorGenReg()

In [2]:
features

tensor([[-0.0070,  0.5044,  1.0000],
        [ 0.6704, -0.3829,  1.0000],
        [ 0.0302,  0.3826,  1.0000],
        ...,
        [-0.9164, -0.6087,  1.0000],
        [ 0.7815,  1.2865,  1.0000],
        [ 1.4819,  1.1390,  1.0000]])

In [7]:
torch.manual_seed(420) 

# 初始化数据
features, labels = tensorGenReg()

# 切分训练集和测试集
Xtrain, Xtest, ytrain, ytest = data_split(features, labels)

# 初始化核心参数
batch_size = 10                                # 小批的数量
lr = 0.03                                      # 学习率
num_epochs = 5                                 # 训练过程遍历几次数据
w = torch.zeros(3, 1, requires_grad = True)    # 随机设置初始权重

# 参与训练的模型方程
net = linreg                                   # 使用回归方程
loss = MSE_loss                                # 均方误差的一半作为损失函数

# 模型训练过程
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, Xtrain, ytrain):
        l = loss(net(X, w), y)
        l.backward()
        sgd(w, lr)

查看训练结果

In [8]:
w

tensor([[ 2.0002],
        [-1.0002],
        [ 0.9996]], requires_grad=True)

查看模型在训练集、测试集上的MSE

In [9]:
MSE_loss(torch.mm(Xtrain, w), ytrain)

tensor(0.0001, grad_fn=<DivBackward0>)

In [10]:
MSE_loss(torch.mm(Xtest, w), ytest)

tensor(9.9141e-05, grad_fn=<DivBackward0>)

至此，我们就完成了一整个从数据集划分，到训练集训练，再到测试集上测试模型性能的一整个流程。

## 四、Dataset和DataLoader基本使用方法与数据集切分函数

&emsp;&emsp;接下来，我们尝试使用PyTorch原生库来实现上述功能，不过这个实现过程略显复杂，首先我们需要了解Dataset和DataLoader的基本使用方法。

### 1.Dataset和DataLoader的基本使用方法

- random_split随机切分函数

&emsp;&emsp;首先，在PyTorch的`torch.utils.data`中，提供了`random_split`函数可用于数据集切分。

In [13]:
from torch.utils.data import random_split

简单测试函数功能

In [14]:
t = torch.arange(12).reshape(4, 3)
t

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])

In [15]:
random_split(t, [2, 2])         # 输入切分的每部分数据集数量

根据生成结果可知，`random_split`函数其实生成了生成器切分结果的生成器，并不是和此前定义的函数一样，直接切分数据后返回。当然这也符合utils.data模块主要生成映射式和迭代式对象的一般规定。

In [16]:
train, test = random_split(t, [2, 2])

使用print函数查看生成器内容

In [17]:
for tr, te in random_split(t, [2, 2]):
    print(tr, te)

tensor([ 9, 10, 11]) tensor([0, 1, 2])
tensor([3, 4, 5]) tensor([6, 7, 8])


- Dataset和Dataloader

&emsp;&emsp;由于在大多数调库建模过程中，我们都是先通过创建Dataset的子类并将数据保存为该子类类型，然后再使用DataLoader进行数据载入，因此更为通用的做法是先利用Dataset和DatasetLoader这两个类进行数据的读取、预处理和载入，然后再使用random_split函数进行切分。      
&emsp;&emsp;再次强调，Dataset类主要负责数据类的生成，在PyTorch中，所有数据集都是Dataset的子类；而DatasetLoader类则是加载模型训练的接口，二者基本使用流程如下：

![49](https://i.loli.net/2021/02/07/LQsueTFV9bDlKx1.jpg)

In [2]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

- 创建数据类

&emsp;&emsp;根据此前描述，PyTorch中所有的数据都是Dataset的子类，换而言之就是在使用PyTorch建模训练数据时，需要创建一个和数据集对应的类来表示该数据集，此前我们使用的TensorDataset函数其实就是一个简单的类型转化函数，将数据统一转化为“TensorDataset”类然后带入模型进行计算。

In [3]:
features, labels = tensorGenReg(bias=False)

In [6]:
features

tensor([[ 0.5846,  0.4064, -0.7022],
        [ 0.5943,  0.5927,  0.8111],
        [-0.4947,  0.2168,  0.0981],
        ...,
        [-0.4568, -1.1319, -1.7560],
        [-1.3112, -0.9356,  1.5156],
        [-1.0726,  0.8814, -1.6092]])

In [7]:
data = TensorDataset(features, labels)

In [8]:
data

而TensorDataset其实使用面较窄，最直接的限制就是该函数只能将张量类型转化为TensorDataset类

In [22]:
TensorDataset([1,2], 1)

AttributeError: 'list' object has no attribute 'size'

In [23]:
TensorDataset?

Init signature: TensorDataset(*args, **kwds)
Docstring:     
Dataset wrapping tensors.

Each sample will be retrieved by indexing tensors along the first dimension.

Arguments:
    *tensors (Tensor): tensors that have the same size of the first dimension.
File:           d:\users\asus\anaconda3\lib\site-packages\torch\utils\data\dataset.py
Type:           type
Subclasses:     


更加通用的数据读取方法则是手动创建一个继承自torch.utils.data.dataset的数据类，用来作为当前数据的表示。例如Lesson 11中的乳腺癌数据，通过如下方式进行读取

In [77]:
from sklearn.datasets import load_breast_cancer as LBC
data = LBC()

简单查看data数据集

In [10]:
data.data          # 返回数据集的特征数组

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [11]:
data.target        # 返回数据集的标签数组  

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [12]:
len(data.data)     # 返回数据集总个数

569

&emsp;&emsp;接下来，创建一个用于表示该数据集的Dataset的子类。在创建Dataset的子类过程中，必须要重写__getitem__方法和__len__方法，其中__getitem__方法返回输入索引后对应的特征和标签，而__len__方法则返回数据集的总数据个数。当然，在必须要进行的__init__初始化过程中，我们也可输入可代表数据集基本属性的相关内容，包括数据集的特征、标签、大小等等，视情况而定。

In [13]:
class LBCDataset(Dataset):
    def __init__(self,data):                       # 创建该类时需要输入sklearn导入的数据集
        self.features = data.data                  # features属性返回数据集特征
        self.labels = data.target                  # labels属性返回数据集标签
        self.lens = len(data.data)                 # lens属性返回数据集大小

    def __getitem__(self, index):
        # 调用该方法时需要输入index数值，方法最终返回index对应的特征和标签
        return self.features[index,:],self.labels[index]    

    def __len__(self):
        # 调用该方法不需要输入额外参数，方法最终返回数据集大小
        return self.lens

In [44]:
data = LBC()
LBC_data = LBCDataset(data)

In [15]:
LBC_data.features

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [16]:
LBC_data.lens

569

In [17]:
# 查看第三条数据
LBC_data.__getitem__(2)

(array([1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
        1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
        4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
        2.250e-02, 4.571e-03, 2.357e+01, 2.553e+01, 1.525e+02, 1.709e+03,
        1.444e-01, 4.245e-01, 4.504e-01, 2.430e-01, 3.613e-01, 8.758e-02]),
 0)

In [18]:
LBC_data.features[2]

array([1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
       1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
       4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
       2.250e-02, 4.571e-03, 2.357e+01, 2.553e+01, 1.525e+02, 1.709e+03,
       1.444e-01, 4.245e-01, 4.504e-01, 2.430e-01, 3.613e-01, 8.758e-02])

In [19]:
LBC_data.labels[2]

0

封装好的数据可以直接进行索引，并且能够返回实体结果

In [49]:
LBC_data[:]

(array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
 

另外，我们可以使用random_split方法对其进行切分

In [31]:
# 确定训练集、测试集大小，此处以7：3划分训练集和测试集
num_train = int(LBC_data.lens * 0.7)
num_test = LBC_data.lens - num_train

In [32]:
num_train            # 训练集个数
num_test             # 测试集个数

398

171

In [33]:
LBC_train, LBC_test = random_split(LBC_data, [num_train, num_test])

注，此时切分的结果是一个映射式的对象，只有dataset和indices两个属性，其中dataset属性用于查看原数据集对象，indices属性用于查看切分后数据集的每一条数据的index（序号）。

In [34]:
LBC_train?

Type:        Subset
String form: <torch.utils.data.dataset.Subset object at 0x0000017B8803D648>
Length:      398
File:        d:\users\asus\anaconda3\lib\site-packages\torch\utils\data\dataset.py
Docstring:  
Subset of a dataset at specified indices.

Arguments:
    dataset (Dataset): The whole Dataset
    indices (sequence): Indices in the whole set selected for subset


In [35]:
LBC_train.dataset

通过切分结果还原原始数据集

In [36]:
LBC_train.dataset == LBC_data      # 还原原数据集

True

在原始数据集中查找切分数据集

In [50]:
LBC_train.indices

[303,
 559,
 341,
 21,
 273,
 535,
 30,
 126,
 283,
 26,
 305,
 135,
 44,
 447,
 481,
 82,
 16,
 386,
 169,
 342,
 513,
 465,
 291,
 69,
 119,
 17,
 231,
 542,
 181,
 52,
 402,
 201,
 356,
 296,
 522,
 6,
 73,
 335,
 553,
 369,
 440,
 89,
 164,
 454,
 114,
 99,
 182,
 455,
 168,
 11,
 340,
 139,
 103,
 317,
 51,
 261,
 354,
 174,
 167,
 540,
 84,
 172,
 337,
 252,
 422,
 334,
 470,
 457,
 360,
 256,
 122,
 322,
 208,
 3,
 390,
 568,
 229,
 410,
 414,
 107,
 555,
 326,
 240,
 377,
 307,
 330,
 347,
 88,
 241,
 339,
 47,
 561,
 204,
 387,
 23,
 541,
 416,
 213,
 62,
 184,
 564,
 428,
 358,
 474,
 391,
 155,
 235,
 398,
 366,
 315,
 185,
 450,
 199,
 80,
 85,
 521,
 348,
 110,
 120,
 385,
 411,
 179,
 502,
 528,
 345,
 464,
 546,
 292,
 519,
 452,
 247,
 140,
 373,
 71,
 444,
 162,
 490,
 234,
 59,
 46,
 98,
 436,
 419,
 15,
 394,
 499,
 251,
 488,
 425,
 287,
 466,
 19,
 471,
 63,
 61,
 83,
 365,
 552,
 70,
 453,
 178,
 156,
 295,
 424,
 197,
 269,
 389,
 417,
 350,
 456,
 448,
 5,
 515,

In [41]:
LBC_train.indices[:10]             # 抽取的训练集数据的index

[384, 475, 64, 490, 496, 136, 123, 300, 175, 342]

当然，无论是迭代式生成数据还是映射式生成数据，都可以使用print查看数据

In [42]:
for i in LBC_train:
    print(i)
    break

(array([1.328e+01, 1.372e+01, 8.579e+01, 5.418e+02, 8.363e-02, 8.575e-02,
       5.077e-02, 2.864e-02, 1.617e-01, 5.594e-02, 1.833e-01, 5.308e-01,
       1.592e+00, 1.526e+01, 4.271e-03, 2.073e-02, 2.828e-02, 8.468e-03,
       1.461e-02, 2.613e-03, 1.424e+01, 1.737e+01, 9.659e+01, 6.237e+02,
       1.166e-01, 2.685e-01, 2.866e-01, 9.173e-02, 2.736e-01, 7.320e-02]), 1)


In [43]:
LBC_data.__getitem__(384)           # 验证是否是LBC_train的第一条数据

(array([1.328e+01, 1.372e+01, 8.579e+01, 5.418e+02, 8.363e-02, 8.575e-02,
        5.077e-02, 2.864e-02, 1.617e-01, 5.594e-02, 1.833e-01, 5.308e-01,
        1.592e+00, 1.526e+01, 4.271e-03, 2.073e-02, 2.828e-02, 8.468e-03,
        1.461e-02, 2.613e-03, 1.424e+01, 1.737e+01, 9.659e+01, 6.237e+02,
        1.166e-01, 2.685e-01, 2.866e-01, 9.173e-02, 2.736e-01, 7.320e-02]),
 1)

In [53]:
LBC_data[LBC_train.indices][1]

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,

> 还是需要强调，虽然PyTorch的数据表示形式会略显复杂，但这是应对复杂大规模数据计算之必须，面对海量、非结构化数据，我们很难去查看一条条数据，而只能通过一些数据集的特性来探索数据信息。

&emsp;&emsp;然后使用DataLoader函数进行数据转化，由一般数据状态转化为“可建模”的状态。所谓“可建模”状态，指的是经过DataLoader处理的数据，不仅包含数据原始的数据信息，还包含数据处理方法信息，如调用几个线程进行训练、分多少批次等，DataLoader常用参数如下：

- batch_size:每次迭代输入多少数据，如果是小批量梯度下降，则输入的数据量就是小批量迭代过程中“小批”的数量      
- shuffle:是否需要先打乱顺序然后再进行小批量的切分，一般训练集需要乱序，而测试集乱序没有意义      
- num_worker:启动多少线程进行计算      


其他更多参数，将随着我们介绍的深入逐步进行介绍

In [44]:
DataLoader?

Init signature: DataLoader(*args, **kwds)
Docstring:     
Data loader. Combines a dataset and a sampler, and provides an iterable over
the given dataset.

The :class:`~torch.utils.data.DataLoader` supports both map-style and
iterable-style datasets with single- or multi-process loading, customizing
loading order and optional automatic batching (collation) and memory pinning.

See :py:mod:`torch.utils.data` documentation page for more details.

Arguments:
    dataset (Dataset): dataset from which to load the data.
    batch_size (int, optional): how many samples per batch to load
        (default: ``1``).
    shuffle (bool, optional): set to ``True`` to have the data reshuffled
        at every epoch (default: ``False``).
    sampler (Sampler or Iterable, optional): defines the strategy to draw
        samples from the dataset. Can be any ``Iterable`` with ``__len__``
        implemented. If specified, :attr:`shuffle` must not be specified.
    batch_sampler (Sampler or Iterable, option

In [45]:
train_loader = DataLoader(LBC_train, batch_size=10, shuffle=True)

In [46]:
test_loader = DataLoader(LBC_test, batch_size=10, shuffle=False)

> 此处需要注意，对于测试集来说，数据装载并不是一定要进行的，如果测试集只是用于检测模型效果，有时可以不用装载直接带入计算。

同样，经过DataLoader处理后的数据也可以使用dataset属性查看原数据

In [48]:
train_loader.dataset

In [49]:
LBC_train

In [50]:
train_loader.dataset == LBC_train

True

> 这里值得一提的是，市面上有很多教材在介绍PyTorch深度学习建模过程中的数据集划分过程，会推荐使用scikit-learn中的train_test_split函数。该函数是可以非常便捷的完成数据集切分，但这种做法只能用于单机运行的数据，并且切分之后还要调用Dataset、DataLoader模块进行数据封装和加载，切分过程看似简单，但其实会额外占用非常多的存储空间和计算资源，当进行超大规模数据训练时，所造成的影响会非常明显（当然，也有可能由于数据规模过大，本地无法运行）。因此，为了更好的适应深度学习真实应用场景，在使用包括数据切分等常用函数时，函数使用优先级是      
<center>Pytorch原生函数和类>依据张量及其常用方法手动创建的函数>Scikit-Learn函数

### 2.建模及评估过程

&emsp;&emsp;接下来，我们尝试通过调库实现完整的数据切分、训练、查看建模结果一整个流程。

- 数据准备过程

In [55]:
# 生成数据
features, labels = tensorGenReg()
features = features[:, :-1]                                  # 剔除最后全是1的列

# 创建一个针对手动创建数据的数据类
class GenData(Dataset):
    def __init__(self, features, labels):           # 创建该类时需要输入的数据集
        self.features = features                    # features属性返回数据集特征
        self.labels = labels                        # labels属性返回数据集标签
        self.lens = len(features)                   # lens属性返回数据集大小

    def __getitem__(self, index):
        # 调用该方法时需要输入index数值，方法最终返回index对应的特征和标签
        return self.features[index,:],self.labels[index]    

    def __len__(self):
        # 调用该方法不需要输入额外参数，方法最终返回数据集大小
        return self.lens

# 实例化对象
data = GenData(features, labels)
    
# 切分数据集
num_train = int(data.lens * 0.7)
num_test = data.lens - num_train
data_train, data_test = random_split(data, [num_train, num_test])

# 加载数据
train_loader = DataLoader(data_train, batch_size=10, shuffle=True)
test_loader = DataLoader(data_test, batch_size=10, shuffle=False)

- 构建模型

In [57]:
# 初始化核心参数
batch_size = 10                                # 小批的数量
lr = 0.03                                      # 学习率
num_epochs = 3                                 # 训练过程遍历几次数据

# Stage 1.定义模型
class LR(nn.Module):
    def __init__(self, in_features=2, out_features=1):       # 定义模型的点线结构
        super(LR, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        
    def forward(self, x):                                    # 定义模型的正向传播规则
        out = self.linear(x)             
        return out

# 实例化模型
LR_model = LR()

# Stage 2.定义损失函数
criterion = nn.MSELoss()

# Stage 3.定义优化方法
optimizer = optim.SGD(LR_model.parameters(), lr = 0.03)

# Stage 4.模型训练与测试
def fit(net, criterion, optimizer, batchdata, epochs=3):
    for epoch  in range(epochs):
        for X, y in batchdata:
            yhat = net.forward(X)
            loss = criterion(yhat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

- 模型训练与测试

In [58]:
fit(net = LR_model,
    criterion = criterion,
    optimizer = optimizer,
    batchdata = train_loader,
    epochs = num_epochs
)

In [59]:
# 查看训练模型
LR_model

LR(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

In [60]:
# 查看模型参数
list(LR_model.parameters())

[Parameter containing:
 tensor([[ 1.9997, -0.9996]], requires_grad=True),
 Parameter containing:
 tensor([1.0005], requires_grad=True)]

查看模型在训练集上表现，首先我们可以通过dataset和indices方法还原训练数据集

In [56]:
data_train.indices            # 返回训练集索引

[705,
 223,
 553,
 386,
 731,
 423,
 867,
 435,
 673,
 469,
 545,
 3,
 11,
 603,
 906,
 522,
 812,
 661,
 119,
 244,
 154,
 543,
 717,
 842,
 894,
 49,
 986,
 825,
 512,
 155,
 652,
 178,
 785,
 479,
 930,
 385,
 277,
 787,
 900,
 871,
 965,
 431,
 711,
 671,
 566,
 870,
 715,
 443,
 557,
 209,
 562,
 666,
 375,
 913,
 340,
 899,
 755,
 968,
 315,
 684,
 162,
 843,
 980,
 838,
 294,
 582,
 493,
 593,
 883,
 125,
 770,
 793,
 525,
 914,
 151,
 319,
 390,
 994,
 266,
 100,
 467,
 212,
 177,
 738,
 929,
 341,
 233,
 882,
 31,
 22,
 145,
 491,
 747,
 243,
 615,
 211,
 451,
 571,
 769,
 416,
 692,
 985,
 239,
 182,
 854,
 694,
 972,
 998,
 561,
 328,
 374,
 485,
 10,
 772,
 283,
 313,
 775,
 488,
 297,
 193,
 506,
 253,
 311,
 886,
 783,
 759,
 539,
 917,
 959,
 258,
 322,
 461,
 323,
 380,
 809,
 613,
 881,
 307,
 548,
 740,
 554,
 241,
 702,
 419,
 441,
 564,
 798,
 997,
 544,
 931,
 542,
 373,
 503,
 518,
 962,
 595,
 703,
 861,
 24,
 459,
 746,
 909,
 183,
 165,
 697,
 8,
 895,
 745,
 9

In [57]:
data[data_train.indices]      # 返回训练集

(tensor([[ 1.1406,  0.4082],
         [ 0.4877,  0.8226],
         [-0.2876,  0.8771],
         ...,
         [ 2.0659,  0.6420],
         [-0.5856,  1.1246],
         [-0.3630, -1.5988]]),
 tensor([[ 2.8812e+00],
         [ 1.1609e+00],
         [-4.6634e-01],
         [ 3.9260e-01],
         [ 4.7393e+00],
         [ 2.6925e-01],
         [-4.2701e+00],
         [-1.9663e+00],
         [ 2.6911e+00],
         [-5.3875e-01],
         [ 4.0773e+00],
         [ 2.5222e+00],
         [ 2.1537e+00],
         [-1.2974e+00],
         [ 1.8957e+00],
         [ 2.6746e+00],
         [ 1.7926e+00],
         [-6.7267e-01],
         [-2.1375e+00],
         [ 8.8927e-01],
         [ 1.2660e+00],
         [ 1.8146e+00],
         [-1.0775e+00],
         [ 1.6035e+00],
         [-5.1649e-01],
         [ 3.4154e+00],
         [-7.7674e-01],
         [-6.4668e-01],
         [ 3.2620e+00],
         [ 2.2912e+00],
         [ 4.4171e-01],
         [ 6.5147e-01],
         [ 1.9962e+00],
         [ 5.4784e

In [58]:
data[data_train.indices][0]      # 返回训练集的特征

tensor([[ 1.1406,  0.4082],
        [ 0.4877,  0.8226],
        [-0.2876,  0.8771],
        ...,
        [ 2.0659,  0.6420],
        [-0.5856,  1.1246],
        [-0.3630, -1.5988]])

In [15]:
# 计算训练集MSE
F.mse_loss(LR_model(data[data_train.indices][0]), data[data_train.indices][1])

tensor(9.4741e-05, grad_fn=<MseLossBackward>)

In [16]:
# 计算测试集MSE
F.mse_loss(LR_model(data[data_test.indices][0]), data[data_test.indices][1])

tensor(0.0001, grad_fn=<MseLossBackward>)

至此，即完成了整个从数据集切分到模型训练，再到查看模型在不同数据集上表现的全过程。

## 五、实用函数补充

&emsp;&emsp;结合上述过程，我们可以补充一些实用函数，方便简化后续建模流程。

- 数据封装、切分和加载函数

&emsp;&emsp;该函数可以直接将输入的特征和标签直接进行封装、切分和加载。该函数可以直接处理此前定义的数据生成器创建的数据。

In [61]:
def split_loader(features, labels, batch_size=10, rate=0.7):
    """数据封装、切分和加载函数：
    
    :param features：输入的特征 
    :param labels: 数据集标签张量
    :param batch_size：数据加载时的每一个小批数据量 
    :param rate: 训练集数据占比
    :return：加载好的训练集和测试集
    """
    data = GenData(features, labels) 
    num_train = int(data.lens * 0.7)
    num_test = data.lens - num_train
    data_train, data_test = random_split(data, [num_train, num_test])
    train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(data_test, batch_size=batch_size, shuffle=False)
    return(train_loader, test_loader)

测试函数性能

In [62]:
# 设置随机数种子
torch.manual_seed(420)   

# 创建数据集
features, labels = tensorGenReg()
features = features[:, :-1]  

# 进行数据加载
train_loader, test_loader = split_loader(features, labels)

In [63]:
# 查看第一条训练集数据
train_loader.dataset[0]

(tensor([-1.4463, -0.6221]), tensor([-1.2863]))

In [66]:
len(train_loader.dataset[:][0])

700

- 模型训练函数

&emsp;&emsp;模型训练函数并不是新的函数，此处正式对其进行定义并写入自定义模块中，方便后续调用。

In [3]:
def fit(net, criterion, optimizer, batchdata, epochs=3, cla=False):
    """模型训练函数
    
    :param net：待训练的模型 
    :param criterion: 损失函数
    :param optimizer：优化算法
    :param batchdata: 训练数据集
    :param cla: 是否是分类问题
    :param epochs: 遍历数据次数
    """
    for epoch  in range(epochs):
        for X, y in batchdata:
            if cla == True:
                y = y.flatten().long()          # 如果是分类问题，需要对y进行整数转化
            yhat = net.forward(X)
            loss = criterion(yhat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

- MSE计算函数

&emsp;&emsp;接下来，我们借助F.mse_loss，定义一个可以直接根据模型输出结果和加载后的数据计算MSE的函数。

In [4]:
def mse_cal(data_loader, net):
    """mse计算函数
    
    :param data_loader：加载好的数据
    :param net: 模型
    :return：根据输入的数据，输出其MSE计算结果
    """
    data = data_loader.dataset                # 还原Dataset类
    X = data[:][0]                            # 还原数据的特征
    y = data[:][1]                            # 还原数据的标签
    yhat = net(X)
    return F.mse_loss(yhat, y)

In [5]:
train_loader.dataset[:][0]

tensor([[-1.4463, -0.6221],
        [-0.4742, -0.2939],
        [ 1.9870,  0.1949],
        ...,
        [-1.6366, -2.1399],
        [-1.8178, -1.4618],
        [ 0.2646,  2.3555]])

接下来，测试函数性能。借助上述建模实验中构建的回归模型，测试函数能否顺利执行。

In [68]:
# 设置随机数种子
torch.manual_seed(420)   

# 实例化模型
LR_model = LR()

# Stage 2.定义损失函数
criterion = nn.MSELoss()

# Stage 3.定义优化方法
optimizer = optim.SGD(LR_model.parameters(), lr = 0.03)

# Stage 4.训练模型
fit(net = LR_model,
    criterion = criterion,
    optimizer = optimizer,
    batchdata = train_loader,
    epochs = 3
)

In [69]:
LR_model

LR(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

In [70]:
mse_cal(train_loader, LR_model)           # 计算训练误差

tensor(0.0001, grad_fn=<MseLossBackward>)

In [71]:
mse_cal(test_loader, LR_model)            # 计算测试误差

tensor(8.8412e-05, grad_fn=<MseLossBackward>)

和F.mse_loss对比

In [72]:
F.mse_loss(LR_model(train_loader.dataset[:][0]), train_loader.dataset[:][1])

tensor(0.0001, grad_fn=<MseLossBackward>)

In [73]:
F.mse_loss(LR_model(test_loader.dataset[:][0]), test_loader.dataset[:][1])

tensor(8.8412e-05, grad_fn=<MseLossBackward>)

- 准确率计算函数

&emsp;&emsp;类似的，定义一个分类问题的准确率计算函数，同样要求输入是加载后的数据集和训练完成的模型。

In [90]:
def accuracy_cal(data_loader, net):
    """准确率
    
    :param data_loader：加载好的数据
    :param net: 模型
    :return：根据输入的数据，输出其准确率计算结果
    """
    data = data_loader.dataset                # 还原Dataset类
    X = data[:][0]                            # 还原数据的特征
    y = data[:][1]                            # 还原数据的标签
    zhat = net(X)                             # 默认是分类问题，并且输出结果是未经softmax转化的结果
    soft_z = F.softmax(zhat, 1)                  # 进行softmax转化
    acc_bool = torch.argmax(soft_z, 1).flatten() == y.flatten()
    acc = torch.mean(acc_bool.float())
    return acc         

In [91]:
t = torch.arange(9).reshape(3, 3).float()
t

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [92]:
F.softmax(t, 1)

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]])

接下来，测试函数性能：

In [93]:
# 设置随机数种子
torch.manual_seed(420)   

# 创建分类数据集
features, labels = tensorGenCla()

# 进行数据加载
train_loader, test_loader = split_loader(features, labels)

In [94]:
class softmaxR(nn.Module):
    def __init__(self, in_features=2, out_features=3, bias=False):       # 定义模型的点线结构
        super(softmaxR, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        
    def forward(self, x):                                    # 定义模型的正向传播规则
        out = self.linear(x)             
        return out

# 实例化模型和
softmax_model = softmaxR()

# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 定义优化算法
optimizer = optim.SGD(softmax_model.parameters(), lr = lr)

# 执行模型训练
fit(net = softmax_model, 
    criterion = criterion, 
    optimizer = optimizer, 
    batchdata = train_loader, 
    epochs = num_epochs, 
    cla=True)

In [96]:
accuracy_cal(train_loader, softmax_model)

tensor(0.8571)

In [98]:
accuracy_cal(test_loader, softmax_model)

tensor(0.8533)

至此，完成本阶段实用函数的添加。